In [31]:
import pickle
from pathlib import Path
import numpy as np
import xarray as xr

In [2]:
output_dir = Path(r"C:\Users\SPEEM\Documents\Python Packages\speem\out") / "fixed"


def fix_data(filepath: Path, new_filepath: Path):
    with open(filepath, "rb") as f:
        raw_data: xr.DataArray = pickle.load(f)
    raw_counts: np.ndarray = raw_data["detector-frame-data"].data
    if raw_counts.ndim == 1:
        raw_counts = np.concatenate(raw_counts)
    else:
        raw_counts = np.concatenate(raw_counts, -1).transpose()
    terminal_voltages = raw_data["power_supply-terminal_voltages-data"].data[0]
    n_counts = raw_counts.shape[0]
    da = xr.DataArray(
        data=raw_counts,
        coords={
            "counts": np.linspace(0, n_counts, n_counts),
            "dim": ["x_px", "y_px", "t_px"],
        },
        dims=("counts", "dim"),
        attrs=terminal_voltages,
    )

    if not new_filepath.parent.exists():
        new_filepath.parent.mkdir()
    da.to_netcdf(new_filepath.with_suffix(".nc"))
    return da, raw_data

In [4]:
filepath = r"C:\Users\SPEEM\Documents\Python Packages\speem\out\data\jgobbo\2025-09-25-BiSe bands pls-0\pickle\raw_daq.pickle"
da, raw_data = fix_data(filepath, output_dir / "2025-09-25" / "01.nc")

In [ ]:
filepath = r"C:\Users\SPEEM\Documents\Python Packages\speem\out\data\jgobbo\2025-09-25-BiSe bands pls-0_09-25-31-323289\pickle\raw_daq.pickle"
with open(filepath, "rb") as f:
    raw = pickle.load(f)

In [ ]:
import numpy as np
import xarray as xr

raw: xr.Dataset
trimmed = raw.copy(deep=True)

for data_var in raw.data_vars:
    dtype = trimmed[data_var].dtype
    # print(f"{data_var} has {dtype=}")
    if dtype == "object":
        # print(trimmed[data_var].data)
        bad_type = not isinstance(trimmed[data_var].data[0], (float, int, np.ndarray))
        # print(bad_type)
        if bad_type:
            print(f"dropping {data_var}")
            trimmed = trimmed.drop_vars(data_var)
        else:
            actual_type = type(trimmed[data_var].data[0])
            print(f"changing {data_var} to {actual_type}")
            trimmed[data_var] = trimmed[data_var].astype(actual_type)

for data_var in trimmed.data_vars:
    print(f"{data_var} - {trimmed[data_var].dtype}")
for data_point in trimmed["detector-frame-data"].data:
    data_point = data_point.astype(float)
# trimmed.to_netcdf(r"C:\Users\SPEEM\Desktop\test.nc")

dropping power_supply-terminal_voltages-data
changing detector-timing_delay-data to <class 'float'>
changing detector-frame_time-data to <class 'float'>
changing phony-stages-0-data to <class 'int'>
changing detector-frame-data to <class 'numpy.ndarray'>
power_supply-terminal_voltages-step - int32
power_supply-terminal_voltages-point - int32
detector-timing_delay-step - int32
detector-timing_delay-point - int32
detector-timing_delay-data - float64
detector-frame_time-step - int32
detector-frame_time-point - int32
detector-frame_time-data - float64
phony-stages-0-step - int32
phony-stages-0-point - int32
phony-stages-0-data - int32
detector-frame-step - int32
detector-frame-point - int32
detector-frame-data - object


ValueError: setting an array element with a sequence.